<a href="https://colab.research.google.com/github/lengochai97/thesis/blob/master/notebooks/feature_construction/22_User_Features_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [0]:
%%capture

import google.colab.drive

google.colab.drive.mount('/content/gdrive', force_remount=True)

# Install Spark and dependencies

In [0]:
import os

os.environ['HADOOP_VERSION'] = '2.7'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/opt/spark'
os.environ['SPARK_VERSION'] = '2.4.3'

In [0]:
%%capture

!wget -qN https://archive.apache.org/dist/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!tar -xzf spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz -C /opt
!rm spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!rm -rf /opt/spark
!ln -s /opt/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION /opt/spark
!pip install -q findspark

# Create SparkSession

In [0]:
import findspark

findspark.init()

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

# Read files

In [0]:
import json

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
DATA_PATH = '/content/gdrive/My Drive/dataset/adressa/one_week'

In [0]:
with open(os.path.join(DATA_PATH, 'schema', 'user_features.json')) as file:
  user_features_schema = T.StructType.fromJson(json.load(file))

In [0]:
df_user_features = spark.read.json(os.path.join(DATA_PATH, 'user_features'), schema=user_features_schema)

# Check results

In [0]:
df_user_features.show(truncate=False)

+-------------------------------------------+----------+----------------+--------------+----------------+--------------+----------------+--------------+----------------+--------------+-------------------+
|userId                                     |time      |userClickCount1H|categoryList1H|userClickCount6H|categoryList6H|userClickCount1D|categoryList1D|userClickCount1W|categoryList1W|categoryListHistory|
+-------------------------------------------+----------+----------------+--------------+----------------+--------------+----------------+--------------+----------------+--------------+-------------------+
|cx:r6211xeqs9o11vu6nnl81a1lk:3lop4czhr7z5g |1483225867|0               |[]            |0               |[]            |0               |[]            |0               |[]            |[]                 |
|cx:38oo4im3dgou42p3yxqoyylf90:3chn0ydngv5vn|1483225995|0               |[]            |0               |[]            |0               |[]            |0               |[]         

## Number of events

In [0]:
df_user_features.count()

1226167

In [0]:
df_user_features.select(F.column('userId'), F.column('time')).distinct().count()

1226167

## Number of users

In [0]:
df_user_features.select(F.column('userId')).distinct().count()

252196

## Disk usage

In [2]:
!du -sh /content/gdrive/My\ Drive/dataset/adressa/one_week/user_features

564M	/content/gdrive/My Drive/dataset/adressa/one_week/user_features
